In [58]:
#Load the data
import dmba
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart
accidents_df = dmba.load_data('accidentsFull.csv')
accidents_df.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0


In [59]:
accidents_df['INJURY'] = np.where(accidents_df['MAX_SEV_IR']>0, 'yes', 'no')
print(accidents_df.INJURY)

0        yes
1         no
2         no
3         no
4         no
        ... 
42178     no
42179    yes
42180     no
42181     no
42182     no
Name: INJURY, Length: 42183, dtype: object


In [60]:
# proportion of "yes" and "no" in the responsevariable "Injury"
print(accidents_df['INJURY'].value_counts() / len(accidents_df))

INJURY
yes    0.5088
no     0.4912
Name: count, dtype: float64


In [61]:
accidents_df.head(12)

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,yes
1,1,2,1,0,0,1,1,3,2,0,...,0,3,2,2,0,0,1,0,0,no
2,1,2,1,0,0,1,0,3,2,0,...,1,2,2,2,0,0,1,0,0,no
3,1,2,1,1,0,0,0,3,2,0,...,1,2,2,1,0,0,1,0,0,no
4,1,1,1,0,0,1,0,3,2,0,...,0,2,3,1,0,0,1,0,0,no
5,1,2,1,1,0,1,0,3,0,0,...,0,2,1,2,1,1,0,0,1,yes
6,1,2,1,0,0,1,1,3,0,0,...,0,2,1,2,0,0,1,0,0,no
7,1,2,1,1,0,1,0,3,0,0,...,0,1,1,1,1,1,0,0,1,yes
8,1,2,1,1,0,1,0,3,0,0,...,0,1,1,2,0,0,1,0,0,no
9,0,2,1,0,0,0,0,3,0,0,...,0,1,1,2,0,0,1,0,0,no


In [62]:
accidents1_df = accidents_df.iloc[0:12, [col_index for col_index, col_name in enumerate(accidents_df.columns) if col_name in ['WEATHER_R', 'TRAF_CON_R', 'INJURY']]]

accidents1_df


,TRAF_CON_R,WEATHER_R,INJURY
0,0,1,yes
1,0,2,no
2,1,2,no
3,1,1,no
4,0,1,no
5,0,2,yes
6,0,2,no
7,0,1,yes
8,0,2,no
9,0,2,no


In [63]:
#change variable types to appropriate ones
accidents1_df.WEATHER_R = accidents1_df.WEATHER_R.astype('category')
accidents1_df.TRAF_CON_R = accidents1_df.TRAF_CON_R.astype('category')
accidents1_df.INJURY = accidents1_df.INJURY.astype('category')
accidents1_df.dtypes

TRAF_CON_R    category
WEATHER_R     category
INJURY        category
dtype: object

In [64]:
#pivot table. This pivot table shows some 'NAN' values as there are no records in accidents1_df
# data with those combinations
accidents1_df.pivot_table(index=['INJURY', 'WEATHER_R'],
                         columns=['TRAF_CON_R'], aggfunc=len)

TRAF_CON_R          0    1    2
INJURY WEATHER_R               
no     1          1.0  1.0  1.0
       2          5.0  1.0  NaN
yes    1          2.0  NaN  NaN
       2          1.0  NaN  NaN

In [65]:
pd.set_option('display.precision',4)
numerator1 = 2/3 * 3/12
denominator1 = 3/12
p1 = numerator1 / denominator1
print(p1)

0.6666666666666666


In [66]:
# P(Injury=yes|WEATHER_R =1, TRAF_CON_R =1)
numerator2 = 0/3 * 3/12
denominator2 = 1/12
p2 = numerator2/denominator2

# P(Injury=yes|WEATHER_R =1, TRAF_CON_R =2)
numerator3 = 0/3 * 3/12
denominator3 = 1/12
p3 = numerator3/denominator3

# P(Injury=yes|WEATHER_R =2, TRAF_CON_R =0)
numerator4 = 1/3 * 3/12
denominator4 = 6/12
p4 = numerator4/denominator4

# P(Injury=yes|WEATHER_R =2, TRAF_CON_R =1)
numerator5 = 0/3 * 3/12
denominator5 = 1/12
p5 = numerator5/denominator5

print('P(Injury=yes | WEATHER_R = 1, TRAF_CON_R=0) =', p1)
print('\nP(Injury=yes | WEATHER_R = 1, TRAF_CON_R =1) =', p2)     
print('\nP(Injury=yes | WEATHER_R = 1, TRAF_CON_R =2) =', p3)
print('\nP(Injury=yes | WEATHER_R = 2, TRAF_CON_R =0) =', p4)
print('\nP(Injury=yes | WEATHER_R = 2, TRAF_CON_R =1) =', p5)
print('\nP(Injury=yes | WEATHER_R = 2, TRAF_CON_R =2) = 0\nIn the above 12 observations there is no observation with (Injury=yes, WEATHER_R =2,TRAF_CON_R =2). The conditional probability here is undefined, since the denominator is zero.')




P(Injury=yes | WEATHER_R = 1, TRAF_CON_R=0) = 0.6666666666666666

P(Injury=yes | WEATHER_R = 1, TRAF_CON_R =1) = 0.0

P(Injury=yes | WEATHER_R = 1, TRAF_CON_R =2) = 0.0

P(Injury=yes | WEATHER_R = 2, TRAF_CON_R =0) = 0.16666666666666666

P(Injury=yes | WEATHER_R = 2, TRAF_CON_R =1) = 0.0

P(Injury=yes | WEATHER_R = 2, TRAF_CON_R =2) = 0
In the above 12 observations there is no observation with (Injury=yes, WEATHER_R =2,TRAF_CON_R =2). The conditional probability here is undefined, since the denominator is zero.


In [67]:
accidents1_df["prob_of_injury"] = [0.667, 0.167, 0, 0, 0.667, 0.167, 0.167, 0.667, 0.167, 0.167, 0.167, 0]
accidents1_df

,TRAF_CON_R,WEATHER_R,INJURY,prob_of_injury
0,0,1,yes,0.667
1,0,2,no,0.167
2,1,2,no,0.000
3,1,1,no,0.000
4,0,1,no,0.667
5,0,2,yes,0.167
6,0,2,no,0.167
7,0,1,yes,0.667
8,0,2,no,0.167
9,0,2,no,0.167


In [68]:
#classification of 12 accidents using these probabilities and a cutoff of 0.5.
accidents1_df['accident'] = ["Yes" if x > 0.5 else "No" for x in accidents1_df['prob_of_injury']]
accidents1_df

,TRAF_CON_R,WEATHER_R,INJURY,prob_of_injury,accident
0,0,1,yes,0.667,Yes
1,0,2,no,0.167,No
2,1,2,no,0.000,No
3,1,1,no,0.000,No
4,0,1,no,0.667,Yes
5,0,2,yes,0.167,No
6,0,2,no,0.167,No
7,0,1,yes,0.667,Yes
8,0,2,no,0.167,No
9,0,2,no,0.167,No


In [69]:
prob = 2/3 * 0/3 * 3/12
prob

0.0

In [70]:
from sklearn.svm import SVC, LinearSVC
#run naive bayes model and obtain probabilities and classification of all 12 records
predictors = ['TRAF_CON_R', 'WEATHER_R']
outcome = 'INJURY'
#fit the model
accidents1_nb = MultinomialNB(alpha=0.01)
accidents1_nb.fit(accidents1_df[predictors], accidents1_df['INJURY'])
#predict probabilities
predProb = accidents1_nb.predict_proba(accidents1_df[predictors])
print('predicted probabilities\n')
print(predProb)
#predict class memberships
print('n\predicted classes\n')
class_pred = accidents1_nb.predict(accidents1_df[predictors])
print(class_pred)

predicted probabilities

[[7.03564216e-01 2.96435784e-01]
 [6.52499618e-01 3.47500382e-01]
 [9.93755543e-01 6.24445695e-03]
 [9.95053326e-01 4.94667443e-03]
 [7.03564216e-01 2.96435784e-01]
 [6.52499618e-01 3.47500382e-01]
 [6.52499618e-01 3.47500382e-01]
 [7.03564216e-01 2.96435784e-01]
 [6.52499618e-01 3.47500382e-01]
 [6.52499618e-01 3.47500382e-01]
 [6.52499618e-01 3.47500382e-01]
 [9.99941348e-01 5.86518326e-05]]
n\predicted classes

['no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no']


In [71]:
# predictors and outcome
predictors = ['HOUR_I_R', 'ALIGN_I', 'WRK_ZONE', 'WKDY_I_R', 'INT_HWY', 'LGTCON_I_R', 'PROFIL_I_R', 'SPD_LIM',
             'SUR_COND', 'TRAF_CON_R', 'TRAF_WAY', 'WEATHER_R']
outcome ='INJURY'
# get dummies
X = pd.get_dummies(accidents_df[predictors])
y = accidents_df['INJURY'].astype('category')
classes = list(y.cat.categories)
#partition the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

In [72]:
# fit the model
accidents_nb = MultinomialNB(alpha=0.01)
accidents_nb.fit(X_train, y_train)
#predict probabilities for training and validation sets
predProb_train = accidents_nb.predict_proba(X_train)
predProb_valid = accidents_nb.predict_proba(X_valid)
# predict class memberships for validation data
y_train_pred = accidents_nb.predict(X_train)
y_valid_pred = accidents_nb.predict(X_valid)

In [73]:
import dmba
from dmba import classificationSummary, gainsChart
# confusion matrix
# training
print('training data\n')
classificationSummary(y_train, y_train_pred, class_names=classes)
# validation
print('\Invalidation data\n')
classificationSummary(y_valid, y_valid_pred, class_names=classes)

training data

Confusion Matrix (Accuracy 0.5291)

       Prediction
Actual   no  yes
    no 4197 8195
   yes 3724 9193
\Invalidation data

Confusion Matrix (Accuracy 0.5288)

       Prediction
Actual   no  yes
    no 2838 5491
   yes 2460 6085


In [74]:
#Overall error for the validation set os 47.12%
error = 1-0.5288
error

0.47119999999999995

In [76]:
improvemnt = 100*(0.4913-0.4712)/0.5087
improvemnt

3.9512482799292323

In [79]:
# consider only required variables
acc_df = accidents_df[['SPD_LIM', 'INJURY']]
#PIVOT TABLE
acc_df.pivot_table(index=['INJURY'],
                 columns=['SPD_LIM'], aggfunc=len)

SPD_LIM,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75
INJURY,,,,,,,,,,,,,,,
no,2,11,93,159,2245,1807,3994,1978,3240,844,3306,727,1371,818,126
yes,4,11,90,92,1960,1908,4547,2326,3347,821,3288,931,1344,636,157
